In [1]:
from scipy.optimize import linprog
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np

## Helper Functions

This section is just the functions that will be used. Scroll down for further instructions.

In [2]:
def isEndOfMonth(date):
    return date.day == date.days_in_month

def convertToTimestamp(settlementDate, maturityDate):
    return pd.to_datetime(settlementDate), pd.to_datetime(maturityDate)

def calculateCouponPayment(couponRate):
    return 100 * (couponRate / 2)

def calculateCashFlow(settlementDate, maturityDate, couponPayment):
    cashFlow = [[maturityDate.date()], [100 + couponPayment]]
    currentDate = maturityDate
    while currentDate > settlementDate:
        currentDate = currentDate - pd.DateOffset(months=6)
        cashFlow[0].append(currentDate.date())
        cashFlow[1].append(couponPayment)
    cashFlow[0] = cashFlow[0][:-1]
    cashFlow[1] = cashFlow[1][:-1]
    return cashFlow

def calculateEndOfMonthCashFlow(settlementDate, maturityDate, couponPayment):
    cashFlow = [[maturityDate.date()], [100 + couponPayment]]
    currentDate = maturityDate
    while currentDate > settlementDate:
        currentDate = currentDate - pd.DateOffset(months=6) + pd.offsets.MonthEnd(0)
        cashFlow[0].append(currentDate.date())
        cashFlow[1].append(couponPayment)
    cashFlow[0] = cashFlow[0][:-1]
    cashFlow[1] = cashFlow[1][:-1]
    return cashFlow

def bondCashFlows(settlementDate: date, maturityDate: date, couponRate: float):
    settlementDate, maturityDate = convertToTimestamp(settlementDate, maturityDate)
    couponPayment = calculateCouponPayment(couponRate)
    if isEndOfMonth(maturityDate):
        return calculateEndOfMonthCashFlow(settlementDate, maturityDate, couponPayment)
    else:
        return calculateCashFlow(settlementDate, maturityDate, couponPayment)
    
def calculateAccruedInterest(settlementDate, couponDate, couponPayment):
    previousCouponDate = couponDate - pd.DateOffset(months=6)
    accruedDays = (settlementDate - previousCouponDate).days
    periodLength = (couponDate - previousCouponDate).days
    return couponPayment * (accruedDays / periodLength)

def calculateEndOfMonthAccruedInterest(settlementDate, couponDate, couponPayment):
    previousCouponDate = couponDate - pd.DateOffset(months=6) + pd.offsets.MonthEnd(0)
    accruedDays = (settlementDate - previousCouponDate).days
    periodLength = (couponDate - previousCouponDate).days
    return couponPayment * (accruedDays / periodLength)

def dirtyPrice(settlementDate: date, maturityDate: date, couponRate: float, cleanPrice: float, type: str):
    if type == 'MARKET BASED BILL':
        return cleanPrice
    settlementDate, maturityDate = convertToTimestamp(settlementDate, maturityDate)
    cashFlows = bondCashFlows(settlementDate, maturityDate, couponRate)
    couponPayment = calculateCouponPayment(couponRate)
    couponDate = pd.to_datetime(cashFlows[0][-1])
    if isEndOfMonth(couponDate):
        accruedInterest = calculateEndOfMonthAccruedInterest(settlementDate, couponDate, couponPayment)
    else:
        accruedInterest = calculateAccruedInterest(settlementDate, couponDate, couponPayment)
    return cleanPrice + accruedInterest

def getBondData(bond_data_file_path):
    bond_data = pd.read_csv(bond_data_file_path, header=None)
    bond_data.columns = ['CUSIP', 'TYPE', 'RATE', 'MATURITY', 'CALL', 'BUY', 'SELL', 'EOD']
    bond_data['MATURITY'] = pd.to_datetime(bond_data['MATURITY'], format='%m/%d/%Y')
    bond_data['CUSIP'] = bond_data['CUSIP'].astype(str)
    bond_data = bond_data[bond_data['TYPE'] != 'TIPS']
    bond_data = bond_data[bond_data['TYPE'] != 'MARKET BASED FRN']
    bond_data = bond_data[bond_data['BUY'] > 0]
    bond_data = bond_data.sort_values(by=['CUSIP'])
    bond_data.reset_index(drop=True, inplace=True)
    return bond_data

def getCFData(cf_data_file_path):
    cf_data = pd.read_csv(cf_data_file_path)
    cf_data['dates'] = pd.to_datetime(cf_data['dates'])
    cf_data.sort_values(by=['dates'], inplace=True)
    cf_data.reset_index(drop=True, inplace=True)
    cf_data['lag_dates'] = cf_data['dates'].shift(1)
    return cf_data

def createZeroPaymentDF(bond_data, cf_data):
    max_date = max(cf_data['dates']).date()
    zero_payment_df = pd.DataFrame({'CUSIP': bond_data['CUSIP'].unique(), 
                                    'dates': [max_date for cusip in range(len(bond_data['CUSIP'].unique()))], 
                                    'payments': [0.0 for cusip in range(len(bond_data['CUSIP'].unique()))]})
    return zero_payment_df

def createPanelData(bond_data, settlementDate):
    dates = bond_data.apply(lambda x: [x['CUSIP'], bondCashFlows(settlementDate=settlementDate, maturityDate=x['MATURITY'], couponRate=x['RATE'])[0]], axis=1)
    payments = bond_data.apply(lambda x: [x['CUSIP'], bondCashFlows(settlementDate=settlementDate, maturityDate=x['MATURITY'], couponRate=x['RATE'])[1]], axis=1)
    dates_df = pd.DataFrame(dates.values.tolist()).explode(1)
    dates_df.columns = ['CUSIP', 'dates']
    payments_df = pd.DataFrame(payments.values.tolist()).explode(1)
    payments_df.columns = ['CUSIP', 'payments']
    joined_df = pd.concat([dates_df, payments_df], axis=1)
    joined_df = joined_df.iloc[:,[0,1,3]]
    joined_df.reset_index(drop=True, inplace=True)
    return joined_df

def getARow(panel_data, zero_payment_df, cf_date, previous_cf_date=None):
    if pd.isnull(previous_cf_date):
        df = panel_data[panel_data['dates'] <= cf_date]
    else:
        df = panel_data[(panel_data['dates'] > previous_cf_date) & (panel_data['dates'] <= cf_date)]
    df = df.sort_values(by=['CUSIP', 'dates'])
    df = pd.concat([df, zero_payment_df], axis=0).reset_index(drop=True)
    A_row = df[['CUSIP', 'payments']].groupby('CUSIP').sum().reset_index().sort_values(by=['CUSIP']).payments.values.tolist()
    return A_row

def buildAMatrix(panel_data, zero_payment_df, cf_data):
    A = np.array(cf_data.apply(lambda x: getARow(panel_data, zero_payment_df, x['dates'].date(), x['lag_dates'].date()), axis=1).values.tolist())
    S = -np.eye(A.shape[0] - 1)
    S = np.hstack((S, np.zeros((S.shape[0], 1))))
    S = np.vstack((S, np.zeros(S.shape[1])))
    L = np.eye(A.shape[0] - 1)
    L = np.hstack((L, np.zeros((L.shape[0], 1))))
    L = np.vstack((np.zeros(L.shape[1]), L))
    S = S + L
    A = np.hstack((A, S))
    return A

def buildBVector(cf_data):
    b = cf_data['cfs'].values / 100
    return b

def buildCVector(bond_data, A, settlementDate):
    c = bond_data.apply(lambda x: dirtyPrice(settlementDate=settlementDate,
                                              maturityDate=x['MATURITY'],
                                              couponRate=x['RATE'],
                                              cleanPrice=x['BUY'],
                                              type=x['TYPE']), axis=1).values.tolist()
    c.extend([0.0 for i in range(A.shape[0])])
    return c

def solveLP(A, b, c):
    result = linprog(c, A_eq=A, b_eq=b)
    return result

def buildSummaryDF(bond_data, result, c):
    summary_df = pd.DataFrame({'CUSIP': bond_data['CUSIP'],
                               'Maturity': bond_data['MATURITY'],
                               'Principal': result.x[:len(bond_data)] * 100 * 100,
                               'Cost': result.x[:len(bond_data)] * 100 * c[:len(bond_data)]})
    summary_df = summary_df[summary_df['Principal'] > 0]
    summary_df['Principal'] = summary_df['Principal'].apply(lambda x: round(x, 0))
    summary_df['Principal'] = summary_df['Principal'].apply(lambda x: '{:,.0f}'.format(x))
    summary_df['Cost'] = summary_df['Cost'].apply(lambda x: round(x))
    summary_df['Cost'] = summary_df['Cost'].apply(lambda x: '{:,.0f}'.format(x))
    summary_df = summary_df.sort_values(by=['Maturity']).reset_index(drop=True)
    return summary_df

def writeSummaryDF(summary_df, output_file_path):
    summary_df.to_csv(output_file_path + '.csv', index=False)
    print('Summary file has been written to: ' + output_file_path + '.csv')

def main(bond_data, cf_data, settlementDate, output_file_path):
    zero_payment_df = createZeroPaymentDF(bond_data, cf_data)
    panel_data = createPanelData(bond_data, settlementDate)
    A = buildAMatrix(panel_data, zero_payment_df, cf_data)
    b = buildBVector(cf_data)
    c = buildCVector(bond_data, A, settlementDate)
    result = solveLP(A, b, c)
    summary_df = buildSummaryDF(bond_data, result, c)
    print('The total cost of the portfolio is: $ {:,.2f}'.format(round(result.fun * 100)))
    writeSummaryDF(summary_df, output_file_path)
    return summary_df

## Dedicated Bond Portfolio Optimizer

Ensure that the desired files are in the same directory for easy of use. If need be, you can enter in the full file path.

First, run the next cell and enter in the file path for the bonds. For example (no quotation marks):

'TreasuryPrices7Feb24.scv' or 'securityprice.csv'

In [14]:
bond_data_file_path = input("Enter the file path for the bond data: ")

try:
    bond_data = getBondData(bond_data_file_path)
    print('Bond data file named', bond_data_file_path, 'loaded successfully.')
except:
    print("Invalid file path. Please rerun cell and try again.")

Bond data file named TreasuryPrices7Feb24.csv loaded successfully.


Now, enter the file path for the cash flows. For example (no quotation marks):

'DatesAndCashFlows1.csv' or 'SingleCFShort.csv'

In [15]:
cf_data_file_path = input("Enter the file path for the cash flow data: ")
try:
    cf_data = getCFData(cf_data_file_path)
    print('Cash flow data file named', cf_data_file_path, 'loaded successfully.')
except:
    print("Invalid file path. Please rerun cell and try again.")

Cash flow data file named DatesAndCashFlows3.csv loaded successfully.


Please enter the settlement date in the form MM/DD/YYYY.

In [16]:
settlement_date = input("Enter the settlement date (mm/dd/yyyy): ")
try:
    settlement_date = datetime.strptime(settlement_date, '%m/%d/%Y')
    print('Settlement date of', settlement_date, 'loaded successfully.')
except:
    print("Invalid date format. Please rerun cell and try again.")

Settlement date of 2024-02-08 00:00:00 loaded successfully.


Lastly, please enter the file name for the output summary. For example (no quotation marks):

'OptimizedResults'

In [6]:
output_file_path = input("Enter the file path for the output: ")

## Run the next cell

In [17]:
summary_df = main(bond_data, cf_data, settlement_date, output_file_path)

The total cost of the portfolio is: $ 102,477,398.00
Summary file has been written to: Test.csv


## Run the next cell to see data frame

In [18]:
summary_df

,CUSIP,Maturity,Principal,Cost
0,912797GP6,2024-02-29,"4,574,437","4,559,733"
1,912797JN8,2024-04-16,"2,959,371","2,929,734"
2,912797HR1,2024-05-23,"1,952,649","1,922,895"
3,912797GB7,2024-07-11,"4,997,773","4,889,429"
4,91282CFG1,2024-08-31,"476,186","477,898"
5,912828YM6,2024-10-31,"1,150,741","1,126,714"
6,912796ZV4,2024-12-26,"461,279","442,179"
7,912797JR9,2025-01-23,"2,497,773","2,386,356"
8,91282CED9,2025-03-15,"4,643,295","4,527,883"
9,91282CFE6,2025-08-15,"3,442,976","3,423,177"
